In [ ]:
#@title Initial Setup and Data Preparation
import tensorflow as tf
from tensorflow.keras.applications import VGG19, InceptionV3, ResNet50, DenseNet201, EfficientNetV2B1
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 50
EPOCHS = 50
LR = 0.001
CLASSES = 11

# Data generators
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2
)

train_data = train_datagen.flow_from_directory(
    'train_data',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    'train_data',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

test_data = test_datagen.flow_from_directory(
    'test_data',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
#@title Model Training Function
def train_and_evaluate(model, model_name):
    print(f"\n{'='*50}")
    print(f"Training {model_name} Model")
    print(f"{'='*50}")

    # Callbacks
    callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True),
        ModelCheckpoint(f'{model_name}_best.h5', save_best_only=True)
    ]

    # Train
    history = model.fit(
        train_data,
        epochs=EPOCHS,
        validation_data=val_data,
        callbacks=callbacks,
        verbose=1
    )

    # Evaluation
    print("\nEvaluating on Test Set...")
    test_results = model.evaluate(test_data, verbose=0)

    # Metrics
    print(f"\nTest Accuracy: {test_results[1]*100:.2f}%")
    print(f"Test Loss: {test_results[0]:.4f}")
    print(f"Precision: {test_results[2]:.4f}")
    print(f"Recall: {test_results[3]:.4f}")

    # Classification Report
    y_pred = model.predict(test_data)
    y_true = test_data.classes
    y_pred_classes = np.argmax(y_pred, axis=1)

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred_classes, target_names=list(test_data.class_indices.keys())))

    # Confusion Matrix
    plt.figure(figsize=(12,10))
    cm = confusion_matrix(y_true, y_pred_classes)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=test_data.class_indices.keys(),
                yticklabels=test_data.class_indices.keys())
    plt.title(f'{model_name} Confusion Matrix')
    plt.show()

    # Training History Plots
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title(f'{model_name} Accuracy')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{model_name} Loss')
    plt.legend()
    plt.show()

    return test_results

In [ ]:
#@title VGG-19 Model Training and Evaluation
def build_vgg19():
    base = VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
    for layer in base.layers[:-4]:  # Freeze all but last 4 layers
        layer.trainable = False

    x = GlobalAveragePooling2D()(base.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(CLASSES, activation='softmax')(x)
    model = Model(inputs=base.input, outputs=predictions)

    model.compile(
        optimizer=Adam(learning_rate=LR),
        loss='categorical_crossentropy',
        metrics=['accuracy', Precision(), Recall()]
    )
    return model

vgg19_model = build_vgg19()
vgg19_results = train_and_evaluate(vgg19_model, "VGG-19")

In [ ]:
#@title GoogleNet (InceptionV3) Model Training and Evaluation
def build_googlenet():
    base = InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))
    for layer in base.layers[:-10]:  # Freeze all but last 10 layers
        layer.trainable = False

    x = GlobalAveragePooling2D()(base.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x)
    predictions = Dense(CLASSES, activation='softmax')(x)
    model = Model(inputs=base.input, outputs=predictions)

    model.compile(
        optimizer=Adam(learning_rate=LR*0.5),  # Lower LR for Inception
        loss='categorical_crossentropy',
        metrics=['accuracy', Precision(), Recall()]
    )
    return model

googlenet_model = build_googlenet()
googlenet_results = train_and_evaluate(googlenet_model, "GoogleNet (InceptionV3)")

In [ ]:
#@title ResNet50 Model Training and Evaluation
def build_resnet50():
    base = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
    for layer in base.layers[:-15]:  # Freeze all but last 15 layers
        layer.trainable = False

    x = GlobalAveragePooling2D()(base.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(CLASSES, activation='softmax')(x)
    model = Model(inputs=base.input, outputs=predictions)

    model.compile(
        optimizer=Adam(learning_rate=LR),
        loss='categorical_crossentropy',
        metrics=['accuracy', Precision(), Recall()]
    )
    return model

resnet50_model = build_resnet50()
resnet50_results = train_and_evaluate(resnet50_model, "ResNet50")

In [ ]:
#@title DenseNet201 Model Training and Evaluation
def build_densenet():
    base = DenseNet201(weights='imagenet', include_top=False, input_shape=(224,224,3))
    for layer in base.layers[:-20]:  # Freeze all but last 20 layers
        layer.trainable = False

    x = GlobalAveragePooling2D()(base.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x)
    predictions = Dense(CLASSES, activation='softmax')(x)
    model = Model(inputs=base.input, outputs=predictions)

    model.compile(
        optimizer=Adam(learning_rate=LR*0.7),  # Adjust LR for DenseNet
        loss='categorical_crossentropy',
        metrics=['accuracy', Precision(), Recall()]
    )
    return model

densenet_model = build_densenet()
densenet_results = train_and_evaluate(densenet_model, "DenseNet201")

In [ ]:
#@title EfficientNetV2-B1 Model Training and Evaluation
def build_efficientnet():
    base = EfficientNetV2B1(weights='imagenet', include_top=False, input_shape=(224,224,3))
    for layer in base.layers[:-5]:  # Freeze all but last 5 layers
        layer.trainable = False

    x = GlobalAveragePooling2D()(base.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    predictions = Dense(CLASSES, activation='softmax')(x)
    model = Model(inputs=base.input, outputs=predictions)

    model.compile(
        optimizer=Adam(learning_rate=LR*1.2),  # Slightly higher LR
        loss='categorical_crossentropy',
        metrics=['accuracy', Precision(), Recall()]
    )
    return model

efficientnet_model = build_efficientnet()
efficientnet_results = train_and_evaluate(efficientnet_model, "EfficientNetV2-B1")

In [ ]:
#@title Final Model Comparison
print("\n\n=== Final Model Comparison ===")
results = {
    'Model': ['VGG19', 'GoogleNet', 'ResNet50', 'DenseNet201', 'EfficientNetV2-B1'],
    'Test Accuracy': [vgg19_results[1], googlenet_results[1], resnet50_results[1],
                     densenet_results[1], efficientnet_results[1]],
    'Test Loss': [vgg19_results[0], googlenet_results[0], resnet50_results[0],
                 densenet_results[0], efficientnet_results[0]],
    'Precision': [vgg19_results[2], googlenet_results[2], resnet50_results[2],
                 densenet_results[2], efficientnet_results[2]],
    'Recall': [vgg19_results[3], googlenet_results[3], resnet50_results[3],
              densenet_results[3], efficientnet_results[3]]
}

import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)

# Visual comparison
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
sns.barplot(x='Model', y='Test Accuracy', data=results_df)
plt.title('Model Accuracy Comparison')
plt.xticks(rotation=45)

plt.subplot(1,2,2)
sns.barplot(x='Model', y='Test Loss', data=results_df)
plt.title('Model Loss Comparison')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Evaluation (Post Training)

In [ ]:
#@title Model Evaluation Pipeline for All 5 Baseline Models
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def evaluate_all_models(models_dict, test_data):
    """
    Evaluates multiple models and generates comparative reports
    Args:
        models_dict: Dictionary of {'model_name': loaded_model}
        test_data: Test data generator (shuffle=False)
    """
    results = []
    class_names = list(test_data.class_indices.keys())

    for name, model in models_dict.items():
        print(f"\n{'='*60}")
        print(f"Evaluating {name}")
        print(f"{'='*60}")

        # Get predictions
        y_true = test_data.classes
        y_pred = model.predict(test_data)
        y_pred_classes = np.argmax(y_pred, axis=1)

        # Classification Report
        print("\nClassification Report:")
        print(classification_report(y_true, y_pred_classes, target_names=class_names, digits=4))

        # Confusion Matrix
        plt.figure(figsize=(12,10))
        cm = confusion_matrix(y_true, y_pred_classes)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names,
                    yticklabels=class_names)
        plt.title(f'{name} Confusion Matrix', fontsize=14)
        plt.xlabel('Predicted Labels')
        plt.ylabel('True Labels')
        plt.xticks(rotation=45)
        plt.yticks(rotation=0)
        plt.show()

        # Store metrics
        f1_per_class = f1_score(y_true, y_pred_classes, average=None)
        macro_f1 = f1_score(y_true, y_pred_classes, average='macro')
        macro_precision = precision_score(y_true, y_pred_classes, average='macro')
        macro_recall = recall_score(y_true, y_pred_classes, average='macro')

        results.append({
            'Model': name,
            'Macro Precision': macro_precision,
            'Macro Recall': macro_recall,
            'Macro F1': macro_f1,
            'Worst Class': class_names[np.argmin(f1_per_class)],
            'Worst F1': np.min(f1_per_class)
        })

        # F1-Score Distribution
        plt.figure(figsize=(10,6))
        sns.barplot(x=f1_per_class, y=class_names, palette='viridis')
        plt.title(f'{name} F1-Scores by Class')
        plt.xlim(0, 1)
        plt.show()

    # Comparative Analysis
    results_df = pd.DataFrame(results)
    print("\n" + "="*60)
    print("Comparative Performance Analysis")
    print("="*60)
    print(results_df.sort_values('Macro F1', ascending=False).to_string(index=False))

    # Visual Comparison
    plt.figure(figsize=(12,6))
    plt.subplot(1,3,1)
    sns.barplot(x='Macro Precision', y='Model', data=results_df, palette='coolwarm')
    plt.title('Macro Precision Comparison')

    plt.subplot(1,3,2)
    sns.barplot(x='Macro Recall', y='Model', data=results_df, palette='coolwarm')
    plt.title('Macro Recall Comparison')

    plt.subplot(1,3,3)
    sns.barplot(x='Macro F1', y='Model', data=results_df, palette='coolwarm')
    plt.title('Macro F1-Score Comparison')

    plt.tight_layout()
    plt.show()

    return results_df

# Load all trained models (replace paths)
models_dict = {
    'VGG19': tf.keras.models.load_model('vgg19_best.keras'),
    'GoogleNet': tf.keras.models.load_model('googlenet_best.keras'),
    'ResNet50': tf.keras.models.load_model('resnet50_best.keras'),
    'DenseNet201': tf.keras.models.load_model('densenet_best.keras'),
    'EfficientNetV2B1': tf.keras.models.load_model('efficientnet_best.keras')
}

# Run evaluation
results_df = evaluate_all_models(models_dict, test_data)

# Save results to CSV
results_df.to_csv('baseline_models_comparison.csv', index=False)
print("\nResults saved to 'baseline_models_comparison.csv'")

# Model Selection

In [ ]:
#@title FPN-Enhanced EfficientNetV2-B1 Model Construction
from tensorflow.keras.layers import Conv2D, UpSampling2D, Concatenate, Input, BatchNormalization
from tensorflow.keras.models import Model

def build_fpn_efficientnet():
    # Input Layer
    input_tensor = Input(shape=(240, 240, 3))  # Slightly larger than 224 for FPN

    # Base Model (EfficientNetV2-B1)
    base_model = EfficientNetV2B1(
        include_top=False,
        weights='imagenet',
        input_tensor=input_tensor
    )

    # Freeze first 75% layers
    for layer in base_model.layers[:int(len(base_model.layers)*0.75)]:
        layer.trainable = False

    # Feature Extraction at Different Scales
    c2 = base_model.get_layer('block2d_add').output  # 120x120
    c3 = base_model.get_layer('block3d_add').output  # 60x60
    c4 = base_model.get_layer('block5d_add').output  # 30x30
    c5 = base_model.get_layer('top_activation').output  # 15x15

    # FPN Top-Down Pathway
    # -------------------
    # P5 (15x15)
    p5 = Conv2D(256, (1, 1), padding='same', name='fpn_c5p5')(c5)

    # P4 (30x30)
    p5_upsampled = UpSampling2D(size=(2, 2), interpolation='bilinear')(p5)
    p4 = Conv2D(256, (1, 1), padding='same', name='fpn_c4p4')(c4)
    p4 = Concatenate(axis=-1)([p5_upsampled, p4])
    p4 = Conv2D(256, (3, 3), padding='same', name='fpn_p4')(p4)
    p4 = BatchNormalization()(p4)

    # P3 (60x60)
    p4_upsampled = UpSampling2D(size=(2, 2), interpolation='bilinear')(p4)
    p3 = Conv2D(256, (1, 1), padding='same', name='fpn_c3p3')(c3)
    p3 = Concatenate(axis=-1)([p4_upsampled, p3])
    p3 = Conv2D(256, (3, 3), padding='same', name='fpn_p3')(p3)
    p3 = BatchNormalization()(p3)

    # P2 (120x120)
    p3_upsampled = UpSampling2D(size=(2, 2), interpolation='bilinear')(p3)
    p2 = Conv2D(256, (1, 1), padding='same', name='fpn_c2p2')(c2)
    p2 = Concatenate(axis=-1)([p3_upsampled, p2])
    p2 = Conv2D(256, (3, 3), padding='same', name='fpn_p2')(p2)
    p2 = BatchNormalization()(p2)

    # Feature Fusion
    # --------------
    # Upsample all features to 60x60 (mid-scale)
    p2_up = UpSampling2D(size=(4, 4), interpolation='bilinear')(p2)
    p3_up = p3  # Already at 60x60
    p4_down = Conv2D(256, (3, 3), strides=2, padding='same')(p4)
    p5_down = Conv2D(256, (3, 3), strides=4, padding='same')(p5)

    fused = Concatenate(axis=-1)([p2_up, p3_up, p4_down, p5_down])
    fused = Conv2D(512, (1, 1), activation='relu')(fused)

    # Classification Head
    # ------------------
    x = GlobalAveragePooling2D()(fused)
    x = Dense(512, activation=None)(x)  # Linear before LRe+SigELU
    x = BatchNormalization()(x)

    # Custom LRe+SigELU Activation
    def lre_sigelu(x):
        from tensorflow.keras import backend as K
        # Trainable parameters
        w1 = 0.6  # ReLU weight (initial)
        w2 = 0.4   # SigELU weight (initial)
        beta = 1.0  # Sigmoid sharpness
        alpha = 1.0 # ELU scale

        # ReLU component
        relu = w1 * K.relu(x)

        # SigELU component
        sig = K.sigmoid(beta * x)
        elu = alpha * (K.exp(x) - 1) * K.cast(x <= 0, 'float32')  # ELU negative part
        sigelu = w2 * sig * elu

        return relu + sigelu

    x = tf.keras.layers.Activation(lre_sigelu)(x)
    x = Dropout(0.3)(x)
    outputs = Dense(11, activation='softmax')(x)

    # Compile Model
    model = Model(inputs=input_tensor, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss='categorical_crossentropy',
        metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
    )

    return model

# Build and Save Model
fpn_model = build_fpn_efficientnet()
fpn_model.summary()

# Save in .keras format
fpn_model.save('fpn_efficientnetv2b1.keras', save_format='keras')
print("Model saved as 'fpn_efficientnetv2b1.keras'")

In [ ]:
#@title Post-Training Model Evaluation Metrics
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def evaluate_model(model, test_data):
    """
    Comprehensive evaluation of a trained model
    Args:
        model: Trained Keras model
        test_data: Test data generator (with shuffle=False)
    """
    # 1. Get predictions and true labels
    y_true = test_data.classes
    y_pred = model.predict(test_data)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # 2. Classification Report
    print("\n" + "="*60)
    print("Detailed Classification Report")
    print("="*60)
    class_names = list(test_data.class_indices.keys())
    report = classification_report(y_true, y_pred_classes, target_names=class_names, digits=4)
    print(report)

    # 3. Confusion Matrix
    plt.figure(figsize=(12,10))
    cm = confusion_matrix(y_true, y_pred_classes)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names,
                yticklabels=class_names)
    plt.title('Confusion Matrix', fontsize=14)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.show()

    # 4. Precision-Recall Curve (Micro-average)
    plt.figure(figsize=(8,6))
    # Convert to binary class indicators for multi-class
    y_true_bin = tf.keras.utils.to_categorical(y_true, num_classes=len(class_names))
    precision = dict()
    recall = dict()
    thresholds = dict()

    for i in range(len(class_names)):
        precision[i], recall[i], thresholds[i] = precision_recall_curve(
            y_true_bin[:, i], y_pred[:, i])
        plt.plot(recall[i], precision[i], lw=2, label=f'{class_names[i]}')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve (Per Class)')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid()
    plt.show()

    # 5. F1-Score Analysis
    f1_per_class = f1_score(y_true, y_pred_classes, average=None)
    f1_df = pd.DataFrame({
        'Class': class_names,
        'F1-Score': f1_per_class
    }).sort_values('F1-Score', ascending=False)

    plt.figure(figsize=(10,6))
    sns.barplot(x='F1-Score', y='Class', data=f1_df, palette='viridis')
    plt.title('F1-Scores by Class')
    plt.xlim(0, 1)
    plt.show()

    # 6. Macro/Micro Averages
    macro_precision = precision_score(y_true, y_pred_classes, average='macro')
    macro_recall = recall_score(y_true, y_pred_classes, average='macro')
    macro_f1 = f1_score(y_true, y_pred_classes, average='macro')

    micro_precision = precision_score(y_true, y_pred_classes, average='micro')
    micro_recall = recall_score(y_true, y_pred_classes, average='micro')
    micro_f1 = f1_score(y_true, y_pred_classes, average='micro')

    print("\n" + "="*60)
    print("Aggregate Metrics Summary")
    print("="*60)
    print(f"Macro Precision: {macro_precision:.4f}")
    print(f"Macro Recall: {macro_recall:.4f}")
    print(f"Macro F1-Score: {macro_f1:.4f}\n")
    print(f"Micro Precision: {micro_precision:.4f}")
    print(f"Micro Recall: {micro_recall:.4f}")
    print(f"Micro F1-Score: {micro_f1:.4f}")

    # 7. Worst Performing Classes
    worst_classes = f1_df.tail(3)
    print("\n" + "="*60)
    print("3 Worst Performing Classes")
    print("="*60)
    print(worst_classes.to_string(index=False))

# Load your trained model (replace with actual path)
model = tf.keras.models.load_model('fpn_efficientnetv2b1.keras')

# Ensure test_data is your test generator (with shuffle=False)
evaluate_model(model, test_data)